In [2]:
import pandas as pd
import numpy as np
import os

In [5]:
print('開始する都道府県コードを入力してください')
start_pref = input()
print(f'開始する都道府県コード : {start_pref}')

print('終了する都道府県コードを入力してください')
finish_pref = input()
print(f'終了する都道府県コード : {finish_pref}')




#浸水深スケーリング
def flood_rank_scaling(column):
    flood_rank = np.where((column == 11) | (column == 21), 1, 
                    np.where((column == 12) | (column == 22), 2, 
                        np.where((column == 13) | (column == 23), 3,
                            np.where((column == 14) | (column == 24) | (column == 25) | (column == 26), 4, 5))))
    return flood_rank
     


#最大―最小スケーリング
def min_max_scaling_numpy(column):
    #指定のカラムをnumpy配列に
    array = column.to_numpy()
    
    #最大―最小スケーリングを実施
    min_val = np.min(array)
    max_val = np.max(array)
    scaled_array = (array - min_val) / (max_val - min_val)
    return scaled_array






def subsidy(burden_df):
    #補助金とそれに対応する移転希望者の割合の辞書を作成
    subsidy_dic = {0.1:0.05,
                   0.2:0.07,
                   0.3:0.08,
                   0.4:0.1,
                   0.5:0.12,
                   0.6:0.14,
                   0.7:0.17,
                   0.8:0.18,
                   0.9:0.19,
                   1.0:0.21}

    # 総和を保存するための空のデータフレームを作成
    subsidy_df = pd.DataFrame(columns=['subsidy_ratio', 'transfer_ratio', 'total_subsidy', 'measures_damage_cost','measures_total_cost', 'without_measures_cost'])

    #補助金毎にループ
    for col, frac in subsidy_dic.items():
        #burden_dfにdamage_priceに対する補助する金額の列を追加
        burden_df[col] = burden_df['house_price']*col


        # データフレームから指定された割合で行をランダムに選ぶ
        sample_df = burden_df[['house_price', 'damage_price', col]].sample(frac=frac)

        # 選択された行に含まれる数値の総和を計算
        total_sum = sample_df[col].sum()
        
        
        #選択された住宅の移転費用と損害額を比較する必要がある．いらない変数といる変数を分ける．
        
        #補助金と損害額の総和
        
        #施策なしの損害額
        without_measures_cost = burden_df['damage_price'].sum()
        #施策有の損害額
        measures_damage_cost = without_measures_cost - sample_df['damage_price'].sum()
        #補助金と損害額の総和
        measures_total_cost = measures_damage_cost + total_sum
        

        # 総和を一時的なデータフレームに保存
        temp_df = pd.DataFrame({
            'subsidy_ratio': [f"subsidy_{col * 100}%"],
            'transfer_ratio': [f"transfer_{frac * 100}%"],
            'total_subsidy': [total_sum],
            'measures_damage_cost':[measures_damage_cost],
            'measures_total_cost':[measures_total_cost],
            'without_measures_cost':[without_measures_cost]
            })

        # 結果をメインのデータフレームに追加
        subsidy_df = pd.concat([subsidy_df, temp_df], ignore_index=True)
        

    return subsidy_df

    










%cd "G:/マイドライブ/akiyamalab/卒論/負担率総集編"
pref = int(start_pref)
while pref <= int(finish_pref):
    
    #市区町村ポリゴンを取得
    total_city_polygon_path = './data/市区町村ポリゴン/japan_ver84/japan_ver84.shp'
    total_city_polygon = gpd.read_file(total_city_polygon_path)
    city_polygon =  total_city_polygon[total_city_polygon["JCODE"].str[:2] == f'{pref:02}']

    
    burden_path = f'./dev/建物家財損害額/{pref}_lost.csv'
    burden_df = pd.read_csv(burden_path, encoding='shift-jis')
    
    
    #市区町村ごとに集計
    for index, row in city_polygon.iterrows():
        city_bldg = gpd.sjoin(burden_df, row['geometry'], op='contain')
    
    
        #浸水ランクが5段階と7段階があるので5段かにスケーリングする
        city_bldg['flood_rank'] = flood_rank_scaling(city_bldg['A31_001'])
        
        #最大―最小スケーリング
        city_bldg['estimated_age_scaled'] = min_max_scaling_numpy(city_bldg['estimated_age'])
        city_bldg['flood_rank_scaled'] = min_max_scaling_numpy(city_bldg['flood_rank'])
    
    
        subsidy(burden_df)
        
        
        # ディレクトリのパスを構築
        directory_path = f'/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/dev/全国/建物数・一階面積/{pref}'
        # ディレクトリが存在しない場合に作成
        os.makedirs(directory_path, exist_ok=True)

        # ファイルパスを構築
        city_bldg_path = f'{directory_path}/{row['JCODE']}_building.csv'
        # CSVとして保存
        city_bldg.to_csv(mesh_build_path, index=False)
    
    pref += 1

開始する都道府県コードを入力してください
開始する都道府県コード : 33
終了する都道府県コードを入力してください
終了する都道府県コード : 33
G:\マイドライブ\akiyamalab\卒論\負担率総集編


In [8]:
import pandas as pd

df = pd.read_csv("G:/マイドライブ/akiyamalab/卒論/負担率総集編/dev/建物家財損害額/33_lost.csv")

In [2]:
import geopandas as gpd

gdf = gpd.read_file("G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/市区町村ポリゴン/japan_ver84/japan_ver84.shp")

In [7]:
gdf['JCODE']

0       01101
1       01102
2       01103
3       01104
4       01105
        ...  
1902    47362
1903    47375
1904    47381
1905    47382
1906      NaN
Name: JCODE, Length: 1907, dtype: object

In [19]:
pref_code = 1

pref_gdf =  gdf[gdf["JCODE"].str[:2] == f'{pref:02}']

for city in pref_gdf['geometry']:
    print(city)

POLYGON ((141.342326939337 43.06681583788077, 141.35520082975472 43.068586666841384, 141.37341055795105 43.06839972117035, 141.38628194555554 43.06952722460193, 141.3896355507403 43.068598054057134, 141.3689558372423 43.061457775156384, 141.36644555062185 43.05796861336097, 141.36274833966434 43.053807505112445, 141.36034749693965 43.04677694619926, 141.35992610880305 43.04285527884821, 141.35609861216142 43.035736386725034, 141.35531250306974 43.0309400000811, 141.3524327758521 43.022883613288286, 141.34593389073666 43.01710777500398, 141.3407152786873 43.02423722516795, 141.3363174967552 43.02809610814904, 141.3281744485921 43.028454999698624, 141.3236697282755 43.02968666710615, 141.31988888486433 43.02924110789098, 141.3194927757712 43.030225279173465, 141.32077693930444 43.03272277473644, 141.32327138796438 43.03349389212957, 141.32360666691432 43.03513110791448, 141.3192505577672 43.03551138714096, 141.31512861229578 43.03381889182856, 141.312477223845 43.03508917072884, 141.3103

In [17]:
pref_gdf

,JCODE,KEN,SICHO,GUN,SEIREI,SIKUCHOSON,CITY_ENG,P_NUM,H_NUM,Shape_Leng,Shape_Area,geometry
0,01101,北海道,石狩振興局,NaN,札幌市,中央区,"Sapporo-shi, Chuo-ku",239944,146024,0.542590,0.005128,"POLYGON ((141.34233 43.06682, 141.35520 43.068..."
1,01102,北海道,石狩振興局,NaN,札幌市,北区,"Sapporo-shi, Kita-ku",286054,153908,0.567773,0.007031,"POLYGON ((141.40839 43.18395, 141.40427 43.182..."
2,01103,北海道,石狩振興局,NaN,札幌市,東区,"Sapporo-shi, Higashi-ku",262298,144421,0.397203,0.006289,"POLYGON ((141.44698 43.15513, 141.44860 43.153..."
3,01104,北海道,石狩振興局,NaN,札幌市,白石区,"Sapporo-shi, Shiroishi-ku",214050,124710,0.308211,0.003815,"POLYGON ((141.46569 43.10000, 141.46812 43.097..."
4,01105,北海道,石狩振興局,NaN,札幌市,豊平区,"Sapporo-shi, Toyohira-ku",224435,129875,0.428316,0.005101,"POLYGON ((141.38479 43.04660, 141.38558 43.047..."
...,...,...,...,...,...,...,...,...,...,...,...,...
189,01696,北海道,根室振興局,国後郡,NaN,泊村,Tomari-mura,0,0,1.920826,0.060048,"MULTIPOLYGON (((145.43677 43.71785, 145.43740 ..."
190,01697,北海道,根室振興局,国後郡,NaN,留夜別村,Ruyobetsu-mura,0,0,2.455546,0.107668,"MULTIPOLYGON (((145.74499 44.07213, 145.74460 ..."
191,01698,北海道,根室振興局,択捉郡,NaN,留別村,Rubetsu-mura,0,0,4.154457,0.168169,"MULTIPOLYGON (((146.98462 44.44315, 146.98317 ..."
192,01699,北海道,根室振興局,紗那郡,NaN,紗那村,Shana-mura,0,0,2.144771,0.108325,"MULTIPOLYGON (((147.95099 44.99498, 147.95177 ..."


In [11]:
A = [
    "0001234",
    "01000",
    "1000",
]
res = [ele.lstrip("0") for ele in A]

In [9]:
df

,tatemon_id,x,y,geometry,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,...,age,dmg_rate,house_price,house_damage_price,assets,children,adult,add_assets,assets_damage_price,total_lost
0,1573,133.797213,34.672247,POINT (133.797212782 34.6722473045),0,166.805,1364,166.805,331012270,1,...,25.5,0.60,4.270208e+07,4.452973e+06,11500000,2,2,4200000,9420000.0,1.387297e+07
1,1688,133.797788,34.673460,POINT (133.79778766 34.6734596316),0,184.710,1364,184.710,331012270,1,...,15.5,0.40,4.728576e+07,9.415541e+06,11500000,2,2,4200000,6280000.0,1.569554e+07
2,1694,133.797064,34.672907,POINT (133.797064016 34.6729072982),0,102.940,1364,102.940,331012270,1,...,5.0,0.40,2.635264e+07,8.833405e+06,11500000,2,4,5800000,6920000.0,1.575340e+07
3,1701,133.797125,34.672804,POINT (133.797124865 34.6728042894),0,68.600,1364,68.600,331012270,1,...,15.5,0.40,1.756160e+07,3.496866e+06,3000000,0,5,4000000,2800000.0,6.296866e+06
4,1703,133.796789,34.672734,POINT (133.79678866 34.6727343175),0,115.070,1364,115.070,331012270,1,...,5.0,0.40,2.945792e+07,9.874295e+06,3000000,0,3,2400000,2160000.0,1.203429e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231777,19890,133.816148,34.860425,POINT (133.816147573 34.8604254275),0,107.045,1364,107.045,33681059005,1,...,15.5,0.40,2.740352e+07,5.456589e+06,5000000,4,2,6800000,4720000.0,1.017659e+07
231778,19898,133.817725,34.860710,POINT (133.817724701 34.8607098307),0,118.770,1364,118.770,33681059006,1,...,5.0,0.40,3.040512e+07,1.019180e+07,11500000,2,2,4200000,6280000.0,1.647180e+07
231779,19908,133.816064,34.861105,POINT (133.816063964 34.861104545),0,168.965,1364,168.965,33681059006,1,...,8.0,0.40,4.325504e+07,1.281733e+07,11500000,2,2,4200000,6280000.0,1.909733e+07
231780,19909,133.816188,34.860946,POINT (133.816187744 34.8609455599),0,181.605,1364,181.605,33681059006,1,...,15.5,0.40,4.649088e+07,9.257264e+06,11500000,2,2,4200000,6280000.0,1.553726e+07


In [50]:
subsidy(burden_df)

,subsidy_ratio,transfer_ratio,total_subsidy,measures_damage_cost,measures_total_cost,without_measures_cost
0,subsidy_30.0%,transfer_64.5%,1.701710e+12,8.050577e+11,2.506768e+12,2.273164e+12
1,subsidy_50.0%,transfer_64.60000000000001%,2.842290e+12,8.045300e+11,3.646820e+12,2.273164e+12
2,subsidy_70.0%,transfer_64.8%,3.989081e+12,8.009014e+11,4.789982e+12,2.273164e+12
3,subsidy_100.0%,transfer_66.8%,5.878401e+12,7.550209e+11,6.633422e+12,2.273164e+12


In [6]:
burden_df

,tatemon_id,x,y,geometry,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,...,house_damage_price,assets,children,adult,add_assets,assets_damage_price,total_lost,flood_rank,estimated_age_scaled,flood_rank_scaled
0,1573,133.797213,34.672247,POINT (133.797212782 34.6722473045),0,166.805,1364,166.805,331012270,1,...,4.452973e+06,11500000,2,2,4200000,9420000.0,1.387297e+07,2,0.500000,0.25
1,1688,133.797788,34.673460,POINT (133.79778766 34.6734596316),0,184.710,1364,184.710,331012270,1,...,9.415541e+06,11500000,2,2,4200000,6280000.0,1.569554e+07,1,0.333333,0.00
2,1694,133.797064,34.672907,POINT (133.797064016 34.6729072982),0,102.940,1364,102.940,331012270,1,...,8.833405e+06,11500000,2,4,5800000,6920000.0,1.575340e+07,1,0.000000,0.00
3,1701,133.797125,34.672804,POINT (133.797124865 34.6728042894),0,68.600,1364,68.600,331012270,1,...,3.496866e+06,3000000,0,5,4000000,2800000.0,6.296866e+06,1,0.333333,0.00
4,1703,133.796789,34.672734,POINT (133.79678866 34.6727343175),0,115.070,1364,115.070,331012270,1,...,9.874295e+06,3000000,0,3,2400000,2160000.0,1.203429e+07,1,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231777,19890,133.816148,34.860425,POINT (133.816147573 34.8604254275),0,107.045,1364,107.045,33681059005,1,...,5.456589e+06,5000000,4,2,6800000,4720000.0,1.017659e+07,1,0.333333,0.00
231778,19898,133.817725,34.860710,POINT (133.817724701 34.8607098307),0,118.770,1364,118.770,33681059006,1,...,1.019180e+07,11500000,2,2,4200000,6280000.0,1.647180e+07,1,0.000000,0.00
231779,19908,133.816064,34.861105,POINT (133.816063964 34.861104545),0,168.965,1364,168.965,33681059006,1,...,1.281733e+07,11500000,2,2,4200000,6280000.0,1.909733e+07,1,0.166667,0.00
231780,19909,133.816188,34.860946,POINT (133.816187744 34.8609455599),0,181.605,1364,181.605,33681059006,1,...,9.257264e+06,11500000,2,2,4200000,6280000.0,1.553726e+07,1,0.333333,0.00


In [3]:
%cd "G:/マイドライブ/akiyamalab/卒論/負担率総集編"
burden_path = f'./dev/損害額/33_step8_1_output.csv'
burden_df = pd.read_csv(burden_path, encoding='shift-jis')

G:\マイドライブ\akiyamalab\卒論\負担率総集編


In [5]:
import numpy as np

# 重みの計算（damage_priceが大きいほど重みが小さくなるように設定）
weights = 1 / (burden_df['damage_price'] + 1)  # 0除算を避けるために1を加える
weights /= weights.sum()  # 重みの合計が1になるように正規化

# データの10%を重みに基づいてランダムに抽出
sample_size = int(len(burden_df) * 0.1)  # 抽出する行数（全体の10%）
sampled_data = burden_df.sample(n=sample_size, weights=weights, random_state=1)

sampled_data.head()  # 抽出されたデータの最初の数行を表示

,tatemon_id,x,y,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,estimated_age,A31_001,cost,dpc_rate,age,dmg_rate,house_price,damage_price
120457,27045,133.948296,34.683880,0,63.220,1364,63.220,331020490,1,7,13,256000,0.036,50.0,0.75,1.618432e+07,6.069120e+05
223911,50700,133.696008,34.629778,0,72.540,1364,72.540,33202209057,1,7,15,256000,0.036,50.0,0.80,1.857024e+07,7.428096e+05
74,9127,133.813706,34.679553,0,132.145,1364,132.145,331012280,1,7,14,256000,0.036,50.0,0.80,3.382912e+07,1.353165e+06
91707,57669,133.908388,34.626896,0,60.885,1364,60.885,331040062,1,4,12,256000,0.036,25.5,0.60,1.558656e+07,1.625366e+06
44246,137710,133.923982,34.666961,0,29.575,1364,29.575,33101040001,1,6,12,256000,0.036,45.5,0.60,7.571200e+06,2.271360e+05


In [7]:
sampled_data

,tatemon_id,x,y,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,estimated_age,A31_001,cost,dpc_rate,age,dmg_rate,house_price,damage_price
120457,27045,133.948296,34.683880,0,63.220,1364,63.220,331020490,1,7,13,256000,0.036,50.0,0.75,1.618432e+07,6.069120e+05
223911,50700,133.696008,34.629778,0,72.540,1364,72.540,33202209057,1,7,15,256000,0.036,50.0,0.80,1.857024e+07,7.428096e+05
74,9127,133.813706,34.679553,0,132.145,1364,132.145,331012280,1,7,14,256000,0.036,50.0,0.80,3.382912e+07,1.353165e+06
91707,57669,133.908388,34.626896,0,60.885,1364,60.885,331040062,1,4,12,256000,0.036,25.5,0.60,1.558656e+07,1.625366e+06
44246,137710,133.923982,34.666961,0,29.575,1364,29.575,33101040001,1,6,12,256000,0.036,45.5,0.60,7.571200e+06,2.271360e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14110,46030,133.870486,34.645637,0,55.510,1364,55.510,331011370,1,7,12,256000,0.036,50.0,0.60,1.421056e+07,4.263168e+05
137673,66883,133.982000,34.647463,0,70.390,1364,70.390,331020240,1,5,13,256000,0.036,35.5,0.75,1.801984e+07,6.757440e+05
144018,50344,133.962812,34.684315,0,70.495,1364,70.495,33102053104,1,5,12,256000,0.036,35.5,0.60,1.804672e+07,5.414016e+05
51634,117066,133.916795,34.640650,0,96.665,1364,96.665,331011300,1,5,11,256000,0.036,35.5,0.40,2.474624e+07,4.949248e+05


In [6]:
weights

0         1.085253e-06
1         3.226271e-07
2         6.556284e-07
3         5.132579e-07
4         5.470825e-07
              ...     
333097    2.480271e-06
333098    8.856450e-07
333099    4.741658e-07
333100    8.936224e-07
333101    3.446658e-07
Name: damage_price, Length: 333102, dtype: float64